In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
PSQL_USER=os.environ["PSQL_USER"]
PSQL_PASSWORD=os.environ["PSQL_PASSWORD"]
MONGODB_URI=os.environ["MONGODB_URI"]
MONGODB_DATABASE=os.environ['MONGODB_DATABASE']
MONGODB_COLLECTION=os.environ['MONGODB_COLLECTION']

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType , StringType
from pyspark.sql.functions import *

In [3]:
# create a local SparkSession
spark = SparkSession.builder\
        .appName("PYSPARK - PostgreSQL to MongoDB")\
        .config("spark.jars.packages","org.mongodb.spark:mongo-spark-connector:10.0.5,org.postgresql:postgresql:42.6.0")\
        .getOrCreate()

Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true
Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true


:: loading settings :: url = jar:file:/home/kali/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/kali/.ivy2/cache
The jars for the packages stored in: /home/kali/.ivy2/jars
org.mongodb.spark#mongo-spark-connector added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-527ce53d-2b22-4a7a-9f53-6b201491c06d;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector;10.0.5 in central
	found org.mongodb#mongodb-driver-sync;4.5.1 in central
	[4.5.1] org.mongodb#mongodb-driver-sync;[4.5.0,4.5.99)
	found org.mongodb#bson;4.5.1 in central
	found org.mongodb#mongodb-driver-core;4.5.1 in central
	found org.postgresql#postgresql;42.6.0 in central
	found org.checkerframework#checker-qual;3.31.0 in central
:: resolution report :: resolve 6397ms :: artifacts dl 8ms
	:: modules in use:
	org.checkerframework#checker-qual;3.31.0 from central in [default]
	org.mongodb#bson;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-core;4.5.1 from central in [default]
	org.mongod

23/04/11 05:37:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
schema=StructType()\
    .add("id",IntegerType())\
    .add("name",StringType())\
    .add("host_id", IntegerType())\
    .add("neighbourhood_cleansed",StringType())\
    .add("price",StringType())

In [5]:
lines=spark\
        .readStream\
        .format('socket')\
        .option('host', 'localhost')\
        .option('port', 9999)\
        .load()\
        .select(from_json(col("value").cast("string"),schema).alias("json"))\
        .select(col("json.*"))

23/04/11 05:37:40 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [6]:
transformation=lines.withColumn('price', regexp_replace('price','([$])',''))\
                    .withColumn("price" , col("price").cast(IntegerType()))\
                    .withColumnRenamed("neighbourhood_cleansed","neighbourhood")

In [7]:
# define a streaming query
dataStreamWriter =transformation.writeStream\
        .format("mongodb")\
        .option("checkpointLocation", "/tmp/pyspark/")\
        .option("forceDeleteTempCheckpointLocation", "true")\
        .option("connection.uri", MONGODB_URI )\
        .option("database", MONGODB_DATABASE )\
        .option("collection", MONGODB_COLLECTION)\
        .trigger(continuous="20 second")\
        .outputMode("append")\
        .start()
        
# run the quer
dataStreamWriter.awaitTermination()

23/04/11 05:38:36 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/04/11 05:38:37 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
23/04/11 05:38:37 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


23/04/11 05:54:38 WARN TextSocketContinuousStream: Stream closed by localhost:9999


ERROR:root:KeyboardInterrupt while sending command.                 (0 + 4) / 4]
Traceback (most recent call last):
  File "/home/kali/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/kali/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [8]:
spark.stop()

23/04/11 05:56:25 WARN NettyRpcEnv: Ignored failure: java.util.concurrent.RejectedExecutionException: Task java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask@33cb6806[Not completed, task = java.util.concurrent.Executors$RunnableAdapter@75991c77[Wrapped task = org.apache.spark.rpc.netty.NettyRpcEnv$$anon$1@2dc9c22e]] rejected from java.util.concurrent.ScheduledThreadPoolExecutor@3a78250e[Terminated, pool size = 0, active threads = 0, queued tasks = 0, completed tasks = 0]
23/04/11 05:56:25 WARN ContinuousQueuedDataReader$DataReaderThread: data reader thread failed
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:87)
	at org.apache.spark.sql.execution.streaming.continuous

Exception in thread "stream execution thread for [id = 2a54a087-2d0c-4394-afb5-b13c29c2547d, runId = 55f14667-1977-46b5-8857-32184d927659]" org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:87)
	at org.apache.spark.sql.execution.streaming.state.StateStoreCoordinatorRef.deactivateInstances(StateStoreCoordinator.scala:119)
	at org.apache.spark.sql.streaming.StreamingQueryManager.notifyQueryTermination(StreamingQueryManager.scala:406)
	at org.apache.spark.sql.execution.streaming.StreamExecution.$anonfun$runStream$3(StreamExecution.scala:357)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.UninterruptibleThread.runUninterruptibly(Uninterrup

23/04/11 05:56:26 ERROR Utils: Aborting task
org.apache.spark.SparkException: Data read failed
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToReadDataError(QueryExecutionErrors.scala:1857)
	at org.apache.spark.sql.execution.streaming.continuous.ContinuousQueuedDataReader.next(ContinuousQueuedDataReader.scala:106)
	at org.apache.spark.sql.execution.streaming.continuous.ContinuousDataSourceRDD$$anon$1.getNext(ContinuousDataSourceRDD.scala:102)
	at org.apache.spark.sql.execution.streaming.continuous.ContinuousDataSourceRDD$$anon$1.getNext(ContinuousDataSourceRDD.scala:94)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.streaming.continuous.ContinuousWriteRDD.$anonfun$compute$1(ContinuousWriteRDD.scala:60)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org